# Section 1
according to the lectures, we define the privacy as the action of removing somethign from the db without affecting the statistics of the query.

## Generate parallel databases

In [1]:
import torch

# the number of entries in our database
num_entries = 5000

db = torch.rand(num_entries) > 0.5
db

tensor([1, 0, 0,  ..., 0, 1, 1], dtype=torch.uint8)

In [2]:
db[0:5]

tensor([1, 0, 0, 0, 1], dtype=torch.uint8)

In [3]:
# Create a funcion to remove element x in db.
def get_parallel_db(db, remove_index):
    return torch.cat((db[0:remove_index], db[remove_index+1:]))

In [4]:
get_parallel_db(db, 3)

tensor([1, 0, 0,  ..., 0, 1, 1], dtype=torch.uint8)

In [5]:
# We are now removing x + 1 elemetns of the db and creating a db with at least all
# all the participants removed.

def get_parallel_dbs(db):
    
    parallel_dbs = []
    
    for i in range(len(db)):
        pdb = get_parallel_db(db,i)
        parallel_dbs.append(pdb)
    return parallel_dbs
            

In [6]:
pdbs = get_parallel_dbs(db)


In [7]:
#  Final function thath condeces all the work.

def create_db_and_parallel(num_entries):

    db = torch.rand(num_entries) > 0.5
    pdbs = get_parallel_dbs(db)
    return db, pdbs

db, pdbs = create_db_and_parallel(30)
    

In [8]:
db

tensor([0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0,
        1, 1, 0, 1, 1, 0], dtype=torch.uint8)

In [9]:
db.shape

torch.Size([30])

In [10]:
len(pdbs)

30

In [11]:
# Section 2

In [12]:
db, pdbs = create_db_and_parallel(30)

In [13]:
def query(db):
    return db.sum()

In [14]:
full_db_result = query(db)
full_db_result

tensor(15)

In [15]:
sensitivity = 0

for pdb in pdbs:
    pdb_result = query(pdb)
    db_distance = torch.abs(pdb_result - full_db_result)
    
    if (db_distance > sensitivity):
        sensitivity = db_distance

In [16]:
sensitivity


tensor(1)

# Sensitivity

The maximun ammount that the query changes  when removing an individual from the datbase is called the **L1 sensitivity** or  **sensitivity**.

In [17]:
def create_db_and_parallel(num_entries):

    db = torch.rand(num_entries) > 0.5
    pdbs = get_parallel_dbs(db)
    return db, pdbs

def get_parallel_dbs(db):
    
    parallel_dbs = []
    
    for i in range(len(db)):
        pdb = get_parallel_db(db,i)
        parallel_dbs.append(pdb)
    return parallel_dbs
         

In [18]:
# P3 - Evaluating the privacy of a function

def sensitivity(query, n_entries=1000):
    db, pdbs = create_db_and_parallel(n_entries)
    
    full_db_result = query(db)
    
    max_distance =0
    
    for pdb in pdbs:
        pdb_result = query(pdb)
        db_distance = torch.abs(pdb_result - full_db_result)
        
        if(db_distance > max_distance):
            max_distance = db_distance
    return max_distance

In [19]:
def query(db):
    return db.float().mean()

In [20]:
sensitivity(query)

tensor(0.0005)

In [21]:
 # Project Calculate L1 for threshhold
    
def query(db, threshold = 5):
    return (db.sum() > threshold)

In [22]:
for i in range(10):
    sens_f = sensitivity(query,n_entries=10)
    print(sens_f)

tensor(255, dtype=torch.uint8)
tensor(255, dtype=torch.uint8)
tensor(255, dtype=torch.uint8)
0
0
0
tensor(255, dtype=torch.uint8)
0
tensor(255, dtype=torch.uint8)
0


In [23]:
# Project: Perform a differential Atack

db, _ = create_db_and_parallel(100)


In [24]:
pdb = get_parallel_db(db, remove_index=10)

db[10]

tensor(0, dtype=torch.uint8)

In [25]:
# diff atack using sum query

sum(db) - sum(pdb)

tensor(0, dtype=torch.uint8)

In [26]:
# diff atack using mean query

(sum(db).float() / len(db)) - (sum(pdb).float()/ len(pdb))

tensor(-0.0046)

In [27]:
# Diff atack using threshold
(sum(db).float() > 49) - (sum(pdb).float()  > 49)

tensor(0, dtype=torch.uint8)

# Section 5

Local diff privacy and Global diff privacy

Local diff pravacy??
Each individual adds noice to their data before sending it to the statistical db

Differential privacy always requires a form of randommness or noise added to the query to protect from things like diff atacks

In [28]:
# Plausible deniability

# Project: Local Diff priv



In [29]:
db, pdbs = create_db_and_parallel(100)



In [30]:
true_result = torch.mean(db.float())
true_result


tensor(0.4400)

In [34]:
augmented_database

tensor([0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 0., 1., 0., 0., 1.,
        1., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
        0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 1.,
        1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
        1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 1., 0.,
        0., 1., 1., 1., 0., 1., 0., 1., 1., 1.])

In [42]:
def query(db):
    
    true_result = torch.mean(db.float())
    
    first_coin_flip = (torch.rand(len(db)) > 0.5).float()
    second_coin_flip = (torch.rand(len(db)) > 0.5).float()
    
    augmented_database = db.float()*first_coin_flip + (1 - first_coin_flip)* second_coin_flip
    db_result = torch.mean(augmented_database.float())*2 - 0.5
    
    return db_result, true_result


In [56]:
db, pdbs = create_db_and_parallel(10000)
query(db)


(tensor(0.4926), tensor(0.5020))

With more dataset , the query goes to the true value fot the corrupted db

### Project 5.  Varying the Amount of Noise

In [59]:
def query(db, noise=0.2):
    
    true_result = torch.mean(db.float())
    
    first_coin_flip = (torch.rand(len(db)) > noise).float()
    second_coin_flip = (torch.rand(len(db)) > 0.5).float()
    
    augmented_database = db.float()*first_coin_flip + (1 - first_coin_flip)* second_coin_flip
    
    sk_result = augmented_database.float().mean()
    
    private_result = ((sk_result/noise) - 0.5 )*noise/(1-noise)
    
    db_result = torch.mean(augmented_database.float())*2 - 0.5
    
    return db_result, true_result


In [69]:
db, pdbs = create_db_and_parallel(100)

private_result, true_result = query(db,noise=0.1)

print(f"With noise: {private_result}")
print(f"With out noise: {true_result}")

With noise: 0.5399999618530273
With out noise: 0.5299999713897705


In [78]:
db, pdbs = create_db_and_parallel(10000)

private_result, true_result = query(db,noise=0.9)

print(f"With noise: {private_result}")
print(f"With out noise: {true_result}")

With noise: 0.4983999729156494
With out noise: 0.4950999915599823


White the point number increases (database size) , you can add more noise

## Global diff privacy

It is a wa to know how much information is leakyng a query

$\epsilon$ and $\delta$ are measure the threshold for the leakage

**When prefer local diff privacy**

When the data is so sensitive, the people are going not to giving to you, prefers *local diff priv* because  the individual data owners want to protect their data before sending it to the thrsted curator.

**Global diff priv** 

When there are most interest in saying " I need the output of this query accurate while still having the same level of privacy"

**How much noise we must add**

Depends on:

* Type of noise (Laplacian / Gaussian)
* Sensitivity of query
* Desired epsilon
* Desired delta


## Project Demo Create a Differentially Private Query

In [82]:
epsilon = 0.5
import numpy as np

In [83]:
db, pdbs = create_db_and_parallel(100)

In [84]:
def sum_query(db):
    return db.sum()


def M(db, query, sensitivity):
    beta = sensitivity / epsilon
    noise = torch.tensor(np.random.laplace(0, beta, 1))
    return query(db) + noise


tensor([0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0,
        1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1,
        1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0,
        1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
        0, 0, 0, 0], dtype=torch.uint8)